In [1]:
import pandas as pd
import spacy
import re
import csv
import json

# Excel Section
## Na razie nie działa bo nie mam pomysłu na zrobienie kolumny labels

In [10]:
# Wczytanie danych z pliku Excel
excel_data = pd.read_excel('CLEAR_corpus_final.xlsx')
# nazwy kolumn, które chcemy zachować
columns_to_keep = ['ID', 'Excerpt']  
# Zachowanie tylko wybranych kolumn
excel_data = excel_data.loc[:, columns_to_keep]
# Dostosowanie nazw kolumn dla danych z excela
excel_data.rename(columns={'ID': 'document', 'Excerpt': 'full_text'}, inplace=True)
excel_data.head()

,document,full_text
0,400,When the young people returned to the ballroom...
1,401,"All through dinner time, Mrs. Fayre was somewh..."
2,402,"As Roger had predicted, the snow departed as q..."
3,403,Mr. Grimes was to come up next morning to Sir ...
4,404,And outside before the palace a great garden w...


In [11]:
def replace_newlines_and_remove_escapes(text):
    # Usunięcie znaków ucieczki przed apostrofami
    text = re.sub(r"\\'", "'", text)
    # Usunięcie znaków ucieczki
    text = re.sub(r'\\', '', text)
    # Zamiana znaków nowej linii na \n\n
    text = re.sub(r'(?<!\\)\n', '\n\n', text)
    return text

# Apply the function to the 'full_text' column
excel_data['full_text'] = excel_data['full_text'].apply(replace_newlines_and_remove_escapes)



In [12]:
# Wczytanie modelu języka angielskiego
nlp = spacy.load('en_core_web_sm')
# Funkcja do tokenizacji tekstu i sprawdzenia, czy token kończy się białym znakiem
def tokenize_and_check_whitespace(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    trailing_whitespace = [token.whitespace_ != '' for token in doc]
    return tokens, trailing_whitespace

# Dodanie kolumn 'tokens' i 'trailing_whitespace'
excel_data[['tokens', 'trailing_whitespace']] = excel_data['full_text'].apply(tokenize_and_check_whitespace).apply(pd.Series)
excel_data.to_json('excel.json', orient='records')

#### pomysły z internetu trzeba się nad tym zastanowić

In [13]:
# def label_entities(text):
#     # Przetworzenie tekstu za pomocą modelu spaCy
#     doc = nlp(text)
#     # Utworzenie listy etykiet
#     labels = ['O'] * len(doc)
#     # Oznaczanie nazw osób
#     for ent in doc.ents:
#         if ent.label_ == 'PERSON':
#             for i in range(ent.start, ent.end):
#                 labels[i] = 'B-NAME_STUDENT' if i == ent.start else 'I-NAME_STUDENT'
#         elif ent.label_ == 'EMAIL':
#             for i in range(ent.start, ent.end):
#                 labels[i] = 'I-EMAIL_STUDENT'
#         elif ent.label_ == 'PHONE_NUM':
#             for i in range(ent.start, ent.end):
#                 labels[i] = 'I-PHONE_NUM_STUDENT'
#         elif ent.label_ == 'URL':
#             for i in range(ent.start, ent.end):
#                 labels[i] = 'I-URL_PERSONAL'
#         elif ent.label_ == 'GPE':
#             for i in range(ent.start, ent.end):
#                 labels[i] = 'I-STREET_ADDRESS'
#         # Dodaj inne etykiety według potrzeb (np. USERNAME, ID_NUM)

#     return labels

# excel_data[['labels']] = excel_data['full_text'].apply(label_entities)
# excel_data.to_json('excel.json', orient='records')

In [14]:
# import spacy
# import pandas as pd

# # Wczytanie modelu spaCy
# nlp = spacy.load('en_core_web_sm')

# def label_entities(text):
#     # Przetworzenie tekstu za pomocą modelu spaCy
#     doc = nlp(text)
#     # Utworzenie listy etykiet
#     labels = ['O'] * len(doc)
#     # Oznaczanie nazw osób, adresów e-mail itp. (jak w poprzednich przykładach)

#     return labels

# # Przykładowy tekst podzielony na słowa

# # Otagowanie słów
# labels = excel_data['full_text'].apply(label_entities)
# print(labels)


# excel_data['labels']  = labels

# # Zapis do pliku JSON
# excel_data.to_json('excel.json', orient='records')


In [ ]:
# from textblob import TextBlob
# import nltk


# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')

# def tag_data(text):
#     blob = TextBlob(text)
#     tagged_text = []

#     for word, pos in blob.tags:
#         if pos == 'NNP':  # NN (rzeczownik, pojedyncza liczba), NNP (rzeczownik, pojedyncza liczba, właściwa nazwa)
#             tagged_text.append((word, 'NAME_STUDENT'))
#         elif '@' in word:
#             tagged_text.append((word, 'USERNAME'))
#         elif '.' in word and '@' in word:
#             tagged_text.append((word, 'EMAIL'))
#         # Dodaj więcej warunków dla innych etykiet, takich jak ID_NUM, PHONE_NUM, URL_PERSONAL itp.

#     return tagged_text


# text = "My name is Szymon Dobrowoslki, email is john.doe@example.com and my student ID is 123456789012."
# tags = tag_data(text)
# print(tags

#### Zapis do formatu `json`

In [15]:
# excel_data.to_json('excel.json', orient='records')

# CSV SECTION

*Koncept*

Plan jest taki żeby przetworzyć dane tak by pasowały do pliku konkursowego. Z racji że dane które znalazłem były podzielone w dość specyficzny sposób, zawierały jedną kolumnę `text` 

Przykład wiersza

`<s>[INST] I need a summary of the book Existential Therapy: 100 Key Points and Techniques for Van Kilback. Can you help? [/INST] I need a summary of the book Existential Therapy: 100 Key Points and Techniques for [FULLNAME]. Can you help? </s>`

stwierdziłem że podziele ję na 2 kolumny `full_text` zgodną z konkursem i tymczasową kolumnę `tagged_text` która posłuży do stworzenia kolumny labels będzie to tekst który zawiera tag(tekst po `[/INST]`)

## Nieprzydatne, zapomniałem o kolumnie labels, przestało działać

#### Wczytanie danych csv, usunięcie oznaczeń, doadanie full_text column

In [2]:
# Wczytanie danych z pliku CSV
csv_data = pd.read_csv('PII-Redaction.csv') 
# Wyrażenie regularne do wydobycia żądanej części tekstu
pattern = r"\[INST\](.*?)\[/INST\]"

# Funkcja do wyodrębnienia żądanej części tekstu z każdego wiersza
def extract_text(text):
    matches = re.findall(pattern, text)
    if matches:
        return matches[0].strip()  # Zwracamy pierwsze dopasowanie, usuwając ewentualne białe znaki na początku i końcu
    else:
        return None

# Wydobycie tekstu z każdego wiersza w kolumnie "full_text"
csv_data['full_text'] = csv_data['text'].apply(extract_text)
csv_data.drop(columns='text')

# Zapisanie zmodyfikowanych danych do pliku CSV - kolumna full_text z usuniętymmi oznaczeniami
csv_data.to_csv('first_mod_csv.csv', index=False)


#### standaryzacja tekstu w full_text

In [3]:
def standardize_csv(input_file, output_file):
    # Wczytaj plik CSV
    data = pd.read_csv(input_file)

    # Ustandaryzuj każdy wiersz
    data['full_text'] = data['full_text'].apply(lambda x: '"' + re.sub(r'^\d+\.\s*', '', x.strip('""').strip()) + '"' if not x.startswith('"') else x)
    # zamiana \ na \n\n ja w pliku konkursowym
    data['full_text'] = data['full_text'].str.replace('\\', '\n\n')
    # Zapisz do pliku CSV w taki sposób żeby nie było w """"""
    data.to_csv(output_file, index=False, quoting=csv.QUOTE_NONE, escapechar='\\')

# Przykładowe użycie funkcji
standardize_csv('first_mod_csv.csv', 'second_mod_csv.csv')

#### dodanie kolumn, document, tokens, trailing_whitespace

In [13]:
# Wczytanie danych z pliku CSV
csv_data = pd.read_csv('second_mod_csv.csv')

# Dodanie kolumny 'document'
csv_data['document'] = range(1, len(csv_data) + 1)

# Wczytanie modelu języka angielskiego
nlp = spacy.load('en_core_web_sm')

# Funkcja do tokenizacji tekstu i sprawdzenia, czy token kończy się białym znakiem
def tokenize_and_check_whitespace(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    trailing_whitespace = [token.whitespace_ != '' for token in doc]
    return tokens, trailing_whitespace

# Dodanie kolumn 'tokens' i 'trailing_whitespace'
csv_data[['tokens', 'trailing_whitespace']] = csv_data['full_text'].apply(tokenize_and_check_whitespace).apply(pd.Series)

csv_data = csv_data[['document','full_text','tokens', 'trailing_whitespace']]

# Zapisanie DataFrame jako plik JSON

csv_data.to_csv('csv_with_added_columns.csv')
csv_data.to_json('csv.json')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\szydo\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\szydo\AppData\Local\Temp\ipykernel_22288\3497299016.py", line 2, in <module>
    csv_data = pd.read_csv('./second_mod_csv.csv')
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\szydo\anaconda3\Lib\site-packages\pandas\io\parsers\readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\szydo\anaconda3\Lib\site-packages\pandas\io\parsers\readers.py", line 583, in _read
    return parser.read(nrows)
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\szydo\anaconda3\Lib\site-packages\pandas\io\parsers\readers.py", line 1704, in read
    ) = self._engine.read(  # type: ignore[attr-defined]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\szyd

#### usunięcie ostatniego elementu w trailing_whitespace ponieważ zliczało miejsce między ."

In [11]:
csv_json_data = pd.read_json('csv.json')
#usuwa ostatni element w trailing_whitespace ponieważ zalicz do zakresu element między znakiem na końcu zdania i " np. ."
csv_json_data['trailing_whitespace'] = csv_json_data['trailing_whitespace'].apply(lambda x: x[:-1] if x[-1] is False else x)
# ponowne zapisanie pliku
csv_json_data.to_json('csv.json', orient='records')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\szydo\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\szydo\AppData\Local\Temp\ipykernel_22288\2694230180.py", line 1, in <module>
    csv_json_data = pd.read_json('csv.json')
                    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\szydo\anaconda3\Lib\site-packages\pandas\io\json\_json.py", line 760, in read_json
    json_reader = JsonReader(
                  ^^^^^^^^^^^
  File "c:\Users\szydo\anaconda3\Lib\site-packages\pandas\io\json\_json.py", line 861, in __init__
    data = self._get_data_from_filepath(filepath_or_buffer)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\szydo\anaconda3\Lib\site-packages\pandas\io\json\_json.py", line 917, in _get_data_from_filepath
    raise FileNotFoundError(f"File {filepath_or_buffer} does not exist")
FileNotFoundError: File csv.json 

## Właściwa implementacja

### Wczytanie danych i stworzenie pliku z tagged_text

In [14]:
# Wczytaj dane z pliku CSV
csv_data_tagged = pd.read_csv('PII-Redaction.csv') 

def extract_text_after_tag(text):
    """funkcja wyodrębnia tekst po tagu [/INST]"""
    if '[/INST]' in text:
        split_text = text.split('[/INST]')
        extracted_text = split_text[1].strip() if len(split_text) > 1 else None
        return extracted_text.replace('</s>', '').strip() if extracted_text else None
    else:
        return text.strip().replace('</s>', '')



# Wyrażenie regularne do wydobycia żądanej części tekstu
pattern = r"\[INST\](.*?)\[/INST\]"
def extract_text(text):
    """wyodrębnienia żądanej części tekstu z każdego wiersza"""
    matches = re.findall(pattern, text)
    if matches:
        return matches[0].strip()  # Zwracamy pierwsze dopasowanie, usuwając ewentualne białe znaki na początku i końcu
    else:
        return None

# Wydobycie tekstu z każdego wiersza do kolumny full_text
csv_data_tagged['full_text'] = csv_data_tagged['text'].apply(extract_text)

# Wydobycie tekstu z każdego wiersza  ztagami do kolumny tagged_text
csv_data_tagged['tagged_text'] = csv_data_tagged['text'].apply(extract_text_after_tag)

# nazwy kolumn, które chcemy zachować
column_to_keep = ['full_text', 'tagged_text']

# Zachowanie tylko wybranej kolumny
csv_data_tagged = csv_data_tagged.loc[:, column_to_keep]

# Zapisanie zmodyfikowanych danych do pliku CSV - kolumna full_text z usuniętymi oznaczeniami
csv_data_tagged.to_csv('full_tag_csv.csv', index=False)


### Standaryzacja tekstu w full_text i tagged_text

In [28]:
# nie działa tak jak planowałem i nie do końca wiem dlaczego w sensie nie zamienia \ na \n\n
def standardize_tagged_csv(input_file, output_file):
    # Wczytaj plik CSV
    data = pd.read_csv(input_file)

    # Ustandaryzuj każdy wiersz
    data['full_text'] = data['full_text'].apply(lambda x: '"' + re.sub(r'^\d+\.\s*', '', x.strip('""').strip()) + '"' if not x.startswith('"') else x)
    data['tagged_text'] = data['tagged_text'].apply(lambda x: '"' + re.sub(r'^\d+\.\s*', '', x.strip('""').strip()) + '"' if not x.startswith('"') else x)
    # zamiana \ na \n\n jak w pliku konkursowym
    data['full_text'] = data['full_text'].str.replace(r'\\', r'\n\n')
    data['tagged_text'] = data['tagged_text'].str.replace(r'\\', r'\n\n')
    # Zapisz do pliku CSV w taki sposób żeby nie było w """"""
    data.to_csv(output_file, index=False, quoting=csv.QUOTE_NONE, escapechar='\n')

# Przykładowe użycie funkcji
standardize_tagged_csv('full_tag_csv.csv', 'full_tag_std_csv.csv')

#### dodanie kolumn, document, tokens, trailing_whitespace

In [29]:
# Wczytanie danych z pliku CSV
csv_data_tagged = pd.read_csv('full_tag_std_csv.csv')

# Dodanie kolumny 'document'
csv_data_tagged['document'] = range(1, len(csv_data_tagged) + 1)

# Wczytanie modelu języka angielskiego
nlp = spacy.load('en_core_web_sm')

# Funkcja do tokenizacji tekstu i sprawdzenia, czy token kończy się białym znakiem
def tokenize_and_check_whitespace(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    trailing_whitespace = [token.whitespace_ != '' for token in doc]
    return tokens, trailing_whitespace

# Dodanie kolumn 'tokens' i 'trailing_whitespace'
csv_data_tagged[['tokens', 'trailing_whitespace']] = csv_data_tagged['full_text'].apply(tokenize_and_check_whitespace).apply(pd.Series)

csv_data_tagged = csv_data_tagged[['document','full_text','tokens', 'trailing_whitespace', 'tagged_text']]



csv_data_tagged.to_csv('csv_tagged_with_added_columns.csv', index=False)
# Zapisanie DataFrame jako plik JSON
csv_data_tagged.to_json('csv_tagged.json')

#### Dodanie labels do pliku z kolumną z tagami

##### Stara wersja

In [30]:
# To nie uwzględniało [FIRSTNAME] [LASTNAME] we właściwy sposób
# def assign_labels(tagged_text):
#     words = tagged_text.split()
#     labels = []
#     for i, word in enumerate(words):
#         if '[' in word:
#             # Wyciągamy tag bez dodatkowych znaków
#             tag = word.split('[')[-1].split(']')[0].upper()
#             if tag == 'FULLNAME':
#                     labels.append("B-NAME-STUDENT")
#                     labels.append("I-NAME-STUDENT") 
#             elif tag in ['FIRSTNAME', 'LASTNAME']:
#                 labels.append("B-NAME-STUDENT")
#             elif tag == 'EMAIL':    
#                 labels.append("B-EMAIL")
#             elif tag in ['USERNAME', 'DISPLAYNAME']:
#                 labels.append("B-USERNAME")
#             else:
#                 labels.append('O')
#         else:
#             labels.append('O')
#     return labels

# # Wczytanie danych z pliku CSV
# csv_data = pd.read_csv('csv_tagged_with_added_columns.csv') 

# # Dodanie kolumny z etykietami
# csv_data['labels'] = csv_data['tagged_text'].apply(assign_labels)

# # Zapisanie zmodyfikowanych danych do pliku CSV
# csv_data.to_csv('csv_tagged_with_labels.csv', index=False)


##### Poprawione

In [31]:
def assign_labels(tagged_text):
    """funkcja do tworzenia listy labeli zgodnie z opisem konkursu, robi to na podstaw"""
    words = tagged_text.split()
    labels = []
    for i, word in enumerate(words):
        if '[' in word:
            # Wyciągamy tag bez dodatkowych znaków
            tag = word.split('[')[-1].split(']')[0].upper()
            if tag == 'FIRSTNAME':
                # Sprawdzamy, czy kolejne tagi po pierwszym tagu imienia to nazwisko
                next_word_index = i + 1
                if next_word_index < len(words) and '[LASTNAME]' in words[next_word_index]:
                    labels.append("B-NAME-STUDENT")
                    labels.append("I-NAME-STUDENT")
                else:
                    labels.append("B-NAME-STUDENT")
            elif tag == 'LASTNAME':
                # Sprawdzamy, czy poprzedni tag to imię
                prev_word_index = i - 1
                if prev_word_index >= 0 and '[FIRSTNAME]' in words[prev_word_index]:
                    continue  # Pomijamy przypisanie etykiety w tym przypadku
                else:
                    labels.append("B-NAME-STUDENT")
            elif tag == 'FULLNAME':
                # Sprawdzamy, czy lista labels ma co najmniej dwa elementy
                    labels.append("B-NAME-STUDENT")
                    labels.append("I-NAME-STUDENT") 
            elif tag == 'EMAIL':    
                labels.append("B-EMAIL")
            elif tag in ['USERNAME', 'DISPLAYNAME']:
                labels.append("B-USERNAME")
            else:
                labels.append('O')
        else:
            labels.append('O')
    return labels

# Wczytanie danych z pliku CSV
csv_data = pd.read_csv('csv_tagged_with_added_columns.csv') 

# Dodanie kolumny z etykietami
csv_data['labels'] = csv_data['tagged_text'].apply(assign_labels)

# Zapisanie zmodyfikowanych danych do pliku CSV
csv_data.to_csv('csv_tagged_with_labels.csv', index=False)

#### Usunięcie kolumny tagged-text, zapis do właściwego formatu `json`

In [32]:
csv_data = pd.read_csv('csv_tagged_with_labels.csv')

csv_data.drop(columns=['tagged_text'], inplace=True)

csv_data.to_csv('final_csv_ready_to_merge.csv', index=False)
csv_data.to_json('csv_almost_ready.json', orient='records')

#### Usunięcie z kolumny `trailing_whitespace` ostatniego `false` ponieważ zlicza łączenie znaków(?/!/.) z "

In [33]:
csv_json_data = pd.read_json('csv_almost_ready.json')
# usuwa ostatnie false z trailing_whitespace
csv_json_data['trailing_whitespace'] = csv_json_data['trailing_whitespace'].apply(lambda x: x[:-1] if x[-1] is False else x)
csv_json_data.to_json('final_csv.json', orient='records')

## Łączenie plików 

### Wczytanie pliku json, sortowanie od największego do najmniejszego

In [34]:
train_data = pd.read_json('train.json')
train_data.sort_values('document', ascending=False)

,document,full_text,tokens,trailing_whitespace,labels
6806,22687,Mind Mapping\n\nChallenge\n\nMy consulting tea...,"[Mind, Mapping, \n\n, Challenge, \n\n, My, con...","[True, False, False, False, False, True, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...,...
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."


### połączenie danych zmiana numeracji

In [35]:
# Wczytanie danych z plików JSON
train_data = pd.read_json('train.json')
csv_data = pd.read_json('final_csv.json')
# excel_data = pd.read_json('excel.json')

# Ustalenie początkowego numeru dokumentu dla danych z plików excel.json i csv.json
next_document_number = train_data['document'].max() + 1

# Aktualizacja numerów dokumentów w danych z plików excel.json i csv.json
# excel_data['document'] = range(next_document_number, next_document_number + len(excel_data))
# csv_data['document'] = range(next_document_number + len(excel_data), next_document_number + len(excel_data) + len(csv_data))
csv_data['document'] = range(next_document_number, next_document_number + len(csv_data))

# Dodanie danych z plików excel.json i csv.json do danych z pliku train.json
# train_data = pd.concat([train_data, excel_data, csv_data], ignore_index=True)
train_data = pd.concat([train_data, csv_data], ignore_index=True)

# Zapisanie połączonych danych do pliku train.json
train_data.to_json('new_train.json', orient='records')
